In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
query_gain_fiber = """
SELECT f4.block_group_code, f4.state, f4.county, f4.perc_urban, f4.population population14, f5.population population15, f6.population population16, f4.avg_n_of_tech70_servs AS service14, f5.avg_n_of_tech70_servs service15, f6.avg_n_of_tech70_servs service16
FROM broadband.fcc_acs_with_geo_and_props_2014 f4
     INNER JOIN broadband.fcc_acs_with_geo_and_props_2015 f5 ON f4.block_group_code = f5.block_group_code
     INNER JOIN broadband.fcc_acs_with_geo_and_props_2016 f6 ON f5.block_group_code = f6.block_group_code
WHERE (f4.avg_n_of_tech70_servs = 0 AND f5.avg_n_of_tech70_servs != 0) OR (f4.avg_n_of_tech70_servs = 0 AND f5.avg_n_of_tech70_servs = 0 AND f6.avg_n_of_tech70_servs != 0)
"""

query_gain_fiber_questionable = """
SELECT f4.block_group_code, f4.state, f4.county, f4.perc_urban, f4.population population14, f5.population population15, f6.population population16, f4.avg_n_of_tech70_servs AS service14, f5.avg_n_of_tech70_servs service15, f6.avg_n_of_tech70_servs service16 
FROM broadband.fcc_acs_with_geo_and_props_2014 f4
     INNER JOIN broadband.fcc_acs_with_geo_and_props_2015 f5 ON f4.block_group_code = f5.block_group_code
     INNER JOIN broadband.fcc_acs_with_geo_and_props_2016 f6 ON f5.block_group_code = f6.block_group_code
WHERE (f4.avg_n_of_tech70_servs = 0 AND f5.avg_n_of_tech70_servs != 0) OR (f5.avg_n_of_tech70_servs = 0 AND f6.avg_n_of_tech70_servs != 0)
"""

df_gain_fiber = pd.read_gbq(query_gain_fiber, project_id='fccdsicapstone-218522', dialect='standard')
df_gain_fiber_questionable = pd.read_gbq(query_gain_fiber_questionable, project_id='fccdsicapstone-218522', dialect='standard')

In [3]:
df_gain_fiber

,block_group_code,state,county,perc_urban,population14,population15,population16,service14,service15,service16
0,120050027042,Florida,Bay County,0.966102,1731,1594,1793,0.0,0.019608,0.019231
1,120050027043,Florida,Bay County,0.744444,2881,2927,3265,0.0,0.019231,0.015873
2,120050026071,Florida,Bay County,0.797619,1388,1070,1008,0.0,0.026316,0.023810
3,120050026051,Florida,Bay County,1.000000,366,422,282,0.0,0.055556,0.055556
4,120050026042,Florida,Bay County,1.000000,3490,3406,3521,0.0,0.037037,0.038462
5,120050027051,Florida,Bay County,0.937500,411,444,465,0.0,0.037037,0.034483
6,120050026062,Florida,Bay County,1.000000,660,803,824,0.0,0.037037,0.037037
7,120050027022,Florida,Bay County,1.000000,860,816,630,0.0,0.040000,0.033333
8,120050027023,Florida,Bay County,0.780000,1201,1289,1303,0.0,0.026316,0.025641
9,120050027053,Florida,Bay County,1.000000,300,298,360,0.0,0.041667,0.037037


In [4]:
len(df_gain_fiber)

16431

In [6]:
len(df_gain_fiber_questionable)

18859

## What count as a block group code that gains a certain type of technology?

Take cable as an example. There are 16431

In [49]:
tech_code = ["10", "11", "12", "20", "30", "40", "41", "42", "43", "50", "70"]
tech_code = ["tech"+x for x in tech_code]

In [36]:
block_gain_fiber = """
SELECT f7.urban_rural
FROM 
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2014
      WHERE file_year = 2014) f4
      RIGHT JOIN
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2015
      WHERE file_year = 2015) f5 ON f4.block_code = f5.block_code
      RIGHT JOIN
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2016
      WHERE file_year = 2016) f6 ON f5.block_code = f6.block_code 
      RIGHT JOIN
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2017
      WHERE file_year = 2017) f7 ON f6.block_code = f7.block_code
WHERE (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs > 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs = 0 AND f5.n_of_[]_servs > 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs IS NULL AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs =0 AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs IS NULL AND f6.n_of_[]_servs IS NULL AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs IS NULL AND f6.n_of_[]_servs = 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs = 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs = 0 AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs = 0 AND f7.n_of_[]_servs > 0)
"""

2651097, 2186547

In [54]:
tech_code_dict_query = """
SELECT *
FROM broadband.tech_code
"""
tech_code_table = pd.read_gbq(tech_code_dict_query, project_id='fccdsicapstone-218522', dialect='standard')

In [71]:
tech_code_name = {}
for i in tech_code_table.iterrows():
    tech_code_name["tech" + str(i[1]["tech_code"])] = i[1]["tech_name"]

In [74]:
urban_rural = list()

for code in tech_code:
    df_block_gain_fiber = pd.read_gbq(block_gain_fiber.replace("[]", code), project_id='fccdsicapstone-218522', dialect='standard')
    ur_count = df_block_gain_fiber["urban_rural"].value_counts()
    urban_rural.append([code, tech_code_name[code],ur_count["U"], ur_count["R"], ur_count["U"]/(ur_count["U"]+ur_count["R"])])    

In [79]:
df_urban_rural = pd.DataFrame(urban_rural, columns = ["Tech_code", "Tech_name", "Urban", "Rural", "Perc_Urban"])

In [81]:
df_urban_rural.to_csv("block_gain_tech.csv")

In [28]:
type(df_block_gain_fiber["urban_rural"].value_counts())

pandas.core.series.Series

In [ ]:

block_gain_fiber = """
SELECT f7.block_code, f7.state, f7.county, f7.urban_rural, f7.urban_type, f7.total_length, f7.land_area, f7.elevation_mean, f4.n_of_[]_servs service14, f5.n_of_[]_servs service15, f6.n_of_[]_servs service16, f7.n_of_[]_servs service17
FROM 
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2014
      WHERE file_year = 2014) f4
      RIGHT JOIN
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2015
      WHERE file_year = 2015) f5 ON f4.block_code = f5.block_code
      RIGHT JOIN
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2016
      WHERE file_year = 2016) f6 ON f5.block_code = f6.block_code 
      RIGHT JOIN
      (SELECT *
      FROM broadband.fcc_allyear_blocklevel_with_geo_agg_2017
      WHERE file_year = 2017) f7 ON f6.block_code = f7.block_code
WHERE (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs > 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs = 0 AND f5.n_of_[]_servs > 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs IS NULL AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs =0 AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs > 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs IS NULL AND f6.n_of_[]_servs IS NULL AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs IS NULL AND f6.n_of_[]_servs = 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs IS NULL AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs = 0 AND f7.n_of_[]_servs > 0)
      OR (f4.n_of_[]_servs = 0 AND f5.n_of_[]_servs = 0 AND f6.n_of_[]_servs = 0 AND f7.n_of_[]_servs > 0)
"""